# Question Answering with LangChain, OpenAI, and MultiQuery Retriever

This interactive workbook demonstrates example of Elasticsearch's [MultiQuery Retriever](https://api.python.langchain.com/en/latest/retrievers/langchain.retrievers.multi_query.MultiQueryRetriever.html) to generate similar queries for a given user input and apply all queries to retrieve a larger set of relevant documents from a vectorstore.

Before we begin, we first split the fictional workplace documents into passages with `langchain` and uses OpenAI to transform these passages into embeddings and then store these into Elasticsearch.

We will then ask a question, generate similar questions using langchain and OpenAI, retrieve relevant passages from the vector store, and use langchain and OpenAI again to provide a summary for the questions.

## Install packages and import modules

In [ ]:
#!python3 -m pip install -qU jq lark langchain langchain-elasticsearch langchain_openai tiktoken

from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_elasticsearch import ElasticsearchStore
from langchain_openai.llms import OpenAI
from langchain.retrievers.multi_query import MultiQueryRetriever
from getpass import getpass
from dotenv import load_dotenv
import os

## Connect to Elasticsearch

ℹ️ We're using an Elastic Cloud deployment of Elasticsearch for this notebook. If you don't have an Elastic Cloud deployment, sign up [here](https://cloud.elastic.co/registration?utm_source=github&utm_content=elasticsearch-labs-notebook) for a free trial. 

We'll use the **Cloud ID** to identify our deployment, because we are using Elastic Cloud deployment. To find the Cloud ID for your deployment, go to https://cloud.elastic.co/deployments and select your deployment.

We will use [ElasticsearchStore](https://api.python.langchain.com/en/latest/vectorstores/langchain.vectorstores.elasticsearch.ElasticsearchStore.html) to connect to our elastic cloud deployment, This would help create and index data easily.  We would also send list of documents that we created in the previous step

In [ ]:
# Cargar variables de entorno
load_dotenv()

# Obtener las variables de entorno
ELASTIC_CLOUD_ID = os.getenv("ELASTIC_CLOUD_ID")
ELASTIC_API_KEY = os.getenv("ELASTIC_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
"""
# https://www.elastic.co/search-labs/tutorials/install-elasticsearch/elastic-cloud#finding-your-cloud-id
ELASTIC_CLOUD_ID = getpass("Elastic Cloud ID ")

# https://www.elastic.co/search-labs/tutorials/install-elasticsearch/elastic-cloud#creating-an-api-key
ELASTIC_API_KEY = getpass("Elastic Api Key: ")

# https://platform.openai.com/api-keys
OPENAI_API_KEY = getpass("OpenAI API key: ")

"""
# Verificar que tenemos todas las claves necesarias
if not OPENAI_API_KEY:
    raise ValueError("No se encontró OPENAI_API_KEY en las variables de entorno")
if not ELASTIC_CLOUD_ID:
    raise ValueError("No se encontró ELASTIC_CLOUD_ID en las variables de entorno")
if not ELASTIC_API_KEY:
    raise ValueError("No se encontró ELASTIC_API_KEY en las variables de entorno")

# Configurar OpenAI
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

# Inicializar embeddings y vectorstore
embeddings = OpenAIEmbeddings()

try:
    vectorstore = ElasticsearchStore(
        es_cloud_id=ELASTIC_CLOUD_ID,
        es_api_key=ELASTIC_API_KEY,
        index_name="my_index", #give it a meaningful name,
        embedding=embeddings,
    )
    print("✅ Conexión exitosa a Elasticsearch")
    print(f"🔗 Conectado al índice: my_index")
except Exception as e:
    print("❌ Error al conectar con Elasticsearch:")
    print(f"Error: {str(e)}")

## Indexing Data into Elasticsearch
Let's download the sample dataset and deserialize the document.

In [ ]:
from urllib.request import urlopen
import json

url = "https://raw.githubusercontent.com/elastic/elasticsearch-labs/main/example-apps/chatbot-rag-app/data/data.json"

response = urlopen(url)
data = json.load(response)

with open("temp.json", "w") as json_file:
    json.dump(data, json_file)

### Split Documents into Passages

We’ll chunk documents into passages in order to improve the retrieval specificity and to ensure that we can provide multiple passages within the context window of the final question answering prompt.

Here we are chunking documents into 800 token passages with an overlap of 400 tokens.

Here we are using a simple splitter but Langchain offers more advanced splitters to reduce the chance of context being lost.

In [ ]:
!pip install -U langchain-community


In [ ]:
from langchain.document_loaders import JSONLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


def metadata_func(record: dict, metadata: dict) -> dict:
    #Populate the metadata dictionary with keys name, summary, url, category, and updated_at.
    # Usar .get() con valores por defecto para manejar campos faltantes
    metadata["name"] = record.get("name", "")
    metadata["summary"] = record.get("summary", "")
    metadata["url"] = record.get("url", "")
    metadata["category"] = record.get("category", "")
    metadata["updated_at"] = record.get("updated_at", "")
    return metadata


# For more loaders https://python.langchain.com/docs/modules/data_connection/document_loaders/
# And 3rd party loaders https://python.langchain.com/docs/modules/data_connection/document_loaders/#third-party-loaders
loader = JSONLoader(
    file_path="temp.json",
    jq_schema=".[]",
    content_key="content",
    metadata_func=metadata_func,
)

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=800, chunk_overlap=400 #define chunk size and chunk overlap
)
docs = loader.load_and_split(text_splitter=text_splitter)

### Bulk Import Passages

Now that we have split each document into the chunk size of 800, we will now index data to elasticsearch using [ElasticsearchStore.from_documents](https://api.python.langchain.com/en/latest/vectorstores/langchain.vectorstores.elasticsearch.ElasticsearchStore.html#langchain.vectorstores.elasticsearch.ElasticsearchStore.from_documents).

We will use Cloud ID, Password and Index name values set in the `Create cloud deployment` step.

In [ ]:
from elasticsearch import Elasticsearch

# Crear el cliente de Elasticsearch
es_client = Elasticsearch(
    cloud_id=ELASTIC_CLOUD_ID,
    api_key=ELASTIC_API_KEY,
)

# Definir el mapeo correcto para el índice
index_mapping = {
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "metadata": {
                "properties": {
                    "name": {"type": "keyword"},
                    "summary": {"type": "text"},
                    "url": {"type": "keyword"},
                    "category": {"type": "keyword"},
                    "updated_at": {"type": "date", "ignore_malformed": True}
                }
            },
            "vector": {
                "type": "dense_vector",
                "dims": 1536,  # Dimensión para embeddings de OpenAI
                "index": True,
                "similarity": "cosine"
            }
        }
    }
}

# Eliminar el índice si existe
if es_client.indices.exists(index="my_index"):
    es_client.indices.delete(index="my_index")

# Crear el índice con el mapeo
es_client.indices.create(index="my_index", body=index_mapping)



ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'my_index'})

In [ ]:
# Inicializar el vectorstore con más parámetros
vectorstore = ElasticsearchStore(
    es_cloud_id=ELASTIC_CLOUD_ID,
    es_api_key=ELASTIC_API_KEY,
    index_name="my_index",
    embedding=embeddings,
    distance_strategy="COSINE",
)

# Intentar indexar los documentos con manejo de errores y configuración correcta
try:
    documents = vectorstore.from_documents(
        docs,
        embeddings,
        index_name="my_index",
        es_cloud_id=ELASTIC_CLOUD_ID,
        es_api_key=ELASTIC_API_KEY,
        bulk_kwargs={
            "timeout": "100s",
            "request_timeout": 100,
            "chunk_size": 50,  # Reducir el tamaño del chunk
            "max_retries": 3
        }
    )
    print("✅ Documentos indexados correctamente")
except Exception as e:
    print(f"❌ Error al indexar documentos: {str(e)}")
    if hasattr(e, 'errors'):
        print("\nDetalles del error:")
        for error in e.errors[:3]:
            print(f"- {error}")

✅ Documentos indexados correctamente


In [ ]:
llm = OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)

retriever = MultiQueryRetriever.from_llm(vectorstore.as_retriever(), llm)

# Question Answering with MultiQuery Retriever

Now that we have the passages stored in Elasticsearch, we can now ask a question to get the relevant passages.

In [ ]:
from langchain.schema.runnable import RunnableParallel, RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.schema import format_document

import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

LLM_CONTEXT_PROMPT = ChatPromptTemplate.from_template(
    """You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Be as verbose and educational in your response as possible. 
    
    context: {context}
    Question: "{question}"
    Answer:
    """
)

LLM_DOCUMENT_PROMPT = PromptTemplate.from_template(
    """
---
SOURCE: {name}
{page_content}
---
"""
)


def _combine_documents(
    docs, document_prompt=LLM_DOCUMENT_PROMPT, document_separator="\n\n"
):
    doc_strings = [format_document(doc, document_prompt) for doc in docs]
    return document_separator.join(doc_strings)


_context = RunnableParallel(
    context=retriever | _combine_documents,
    question=RunnablePassthrough(),
)

chain = _context | LLM_CONTEXT_PROMPT | llm

ans = chain.invoke("what is the nasa sales team?")

print("---- Answer ----")
print(ans)

INFO:langchain.retrievers.multi_query:Generated queries: ['1. Can you provide information on the sales team at NASA?', '2. How does the sales team operate within NASA?', '3. What are the responsibilities of the NASA sales team?']


---- Answer ----
The NASA sales team is a part of the Americas region in the sales organization of the company. It is led by two Area Vice-Presidents, Laura Martinez for North America and Gary Johnson for South America. The team is responsible for promoting and selling the company's products and services in the North and South American markets. They work closely with other departments, such as marketing, product development, and customer support, to ensure the company's success in these regions.


**Generate at least two new iteratioins of the previous cells - Be creative.** Did you master Multi-
Query Retriever concepts through this lab?

In [ ]:
# Primera iteración - Políticas de empresa
from langchain.schema.runnable import RunnableParallel, RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.schema import format_document

import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

# Prompt template para políticas
POLICY_PROMPT = ChatPromptTemplate.from_template(
    """Eres un asistente especializado en políticas empresariales. 
    Utiliza el siguiente contexto para responder la pregunta de manera detallada y profesional.
    Si no encuentras la información específica, indícalo claramente.
    
    Contexto: {context}
    Pregunta: "{question}"
    Respuesta:
    """
)

# Configuración del chain
_context = RunnableParallel(
    context=retriever | _combine_documents,
    question=RunnablePassthrough(),
)

policy_chain = _context | POLICY_PROMPT | llm

# Probar con una pregunta sobre políticas
print("🔍 Consultando políticas de trabajo remoto...")
response = policy_chain.invoke("¿Cuáles son las políticas de trabajo remoto en la empresa?")
print("\n📋 Respuesta:")
print(response)

🔍 Consultando políticas de trabajo remoto...


INFO:langchain.retrievers.multi_query:Generated queries: ['1. ¿Qué medidas se han implementado en la empresa para facilitar el trabajo remoto?', '2. ¿Cómo ha adaptado la empresa sus políticas de trabajo ante la situación actual?', '3. ¿Existen políticas específicas para el trabajo remoto en la empresa y cuáles son?']



📋 Respuesta:

La política de trabajo remoto de nuestra empresa se implementó en marzo de 2020 con el propósito de proporcionar pautas y apoyo a los empleados para realizar su trabajo de manera remota durante la pandemia de COVID-19 y en el futuro. Esta política se aplica a todos los empleados elegibles, quienes deben obtener la aprobación de su supervisor directo y el departamento de recursos humanos para trabajar desde casa. Se proporcionará el equipo y los recursos necesarios para el trabajo remoto, y se espera que los empleados mantengan una comunicación efectiva con sus supervisores y colegas. También se espera que los empleados mantengan el mismo nivel de rendimiento y productividad que si estuvieran trabajando en la oficina. Además, se deben seguir las políticas de confidencialidad y seguridad de datos, y se alienta a los empleados a priorizar su salud y bienestar mientras trabajan desde casa. Esta política se revisará periódicamente y se actualizará según sea necesario. Si tien

In [ ]:
# Segunda iteración - Especificaciones técnicas
# Prompt template para análisis técnico
TECHNICAL_PROMPT = ChatPromptTemplate.from_template(
    """Eres un experto técnico. Analiza la siguiente información y proporciona una respuesta 
    detallada y técnicamente precisa. Incluye especificaciones cuando sea posible.
    
    Información disponible: {context}
    Consulta técnica: "{question}"
    
    Análisis técnico:
    """
)

# Configuración del chain técnico
technical_chain = _context | TECHNICAL_PROMPT | llm

# Función para realizar múltiples consultas técnicas
def analyze_technical_aspects(product_name: str):
    print(f"🔬 Análisis técnico completo para: {product_name}")
    print("=" * 50)
    
    # Lista de aspectos técnicos a consultar
    aspects = [
        f"¿Cuáles son las especificaciones técnicas de {product_name}?",
        f"¿Qué requisitos de sistema tiene {product_name}?",
        f"¿Cuáles son las características de seguridad de {product_name}?"
    ]
    
    for aspect in aspects:
        print(f"\n📊 Consultando: {aspect}")
        response = technical_chain.invoke(aspect)
        print(f"Resultado:\n{response}\n")
        print("-" * 30)

# Probar el análisis técnico
analyze_technical_aspects("el sistema de gestión de documentos")

🔬 Análisis técnico completo para: el sistema de gestión de documentos

📊 Consultando: ¿Cuáles son las especificaciones técnicas de el sistema de gestión de documentos?


INFO:langchain.retrievers.multi_query:Generated queries: ['1. ¿Qué características técnicas tiene el sistema de gestión de documentos?', '2. ¿Puedes darme detalles sobre las especificaciones técnicas del sistema de gestión de documentos?', '3. ¿Cuáles son las especificaciones técnicas más importantes del sistema de gestión de documentos?']


Resultado:

Las especificaciones técnicas de un sistema de gestión de documentos pueden variar dependiendo del proveedor y las necesidades específicas de la empresa. Sin embargo, algunas de las características y requisitos comunes incluyen:

1. Almacenamiento en la nube: El sistema debe permitir el almacenamiento de documentos en la nube para facilitar el acceso y la colaboración en tiempo real.

2. Seguridad: El sistema debe contar con medidas de seguridad robustas para proteger la información confidencial y evitar accesos no autorizados.

3. Integración con otros sistemas: Es importante que el sistema pueda integrarse con otros sistemas utilizados en la empresa, como el correo electrónico o el sistema de gestión de clientes.

4. Control de versiones: El sistema debe permitir la gestión de versiones de los documentos, para poder realizar un seguimiento de los cambios y revertir a versiones anteriores si es necesario.

5. Búsqueda avanzada: El sistema debe contar con una función de bús

INFO:langchain.retrievers.multi_query:Generated queries: ['1. ¿Cuáles son los requerimientos de hardware para el sistema de gestión de documentos?', '2. ¿Qué especificaciones técnicas son necesarias para el sistema de gestión de documentos?', '3. ¿Cuáles son los componentes necesarios para el sistema de gestión de documentos?']


Resultado:

El sistema de gestión de documentos puede tener diferentes requisitos de sistema dependiendo de su funcionalidad y características específicas. Sin embargo, algunos requisitos comunes pueden incluir:

1. Sistema operativo: El sistema de gestión de documentos puede requerir un sistema operativo específico, como Windows, Mac o Linux, para funcionar correctamente.

2. Espacio de almacenamiento: Dependiendo del tamaño y la cantidad de documentos que se gestionen, el sistema puede requerir un cierto espacio de almacenamiento en disco para guardar los archivos.

3. Memoria RAM: El sistema puede requerir una cantidad mínima de memoria RAM para funcionar de manera eficiente y procesar grandes cantidades de datos.

4. Procesador: Un procesador rápido y potente puede ser necesario para manejar grandes cantidades de documentos y realizar tareas de búsqueda y organización de manera eficiente.

5. Conexión a internet: Si el sistema de gestión de documentos es basado en la nube, puede re

INFO:langchain.retrievers.multi_query:Generated queries: ['1. ¿Qué medidas de seguridad se implementan en el sistema de gestión de documentos?', '2. ¿Cómo se protegen los documentos en el sistema de gestión de documentos?', '3. ¿Cuáles son las funciones de seguridad incorporadas en el sistema de gestión de documentos?']


Resultado:

El sistema de gestión de documentos es una herramienta esencial para cualquier empresa u organización, ya que permite almacenar, organizar y gestionar de manera eficiente y segura toda la información y documentos importantes. Por lo tanto, es importante que este sistema cuente con características de seguridad sólidas para proteger la información confidencial y sensible de la empresa.

Algunas de las características de seguridad que debe tener un sistema de gestión de documentos son:

1. Acceso restringido: El sistema debe permitir restringir el acceso a ciertos documentos o carpetas solo a usuarios autorizados. Esto se puede lograr mediante la asignación de permisos y roles a cada usuario, lo que garantiza que solo puedan acceder a la información que necesitan para realizar sus tareas.

2. Autenticación de usuarios: Es importante que el sistema cuente con un sistema de autenticación sólido para garantizar que solo los usuarios autorizados puedan acceder al sistema. Esto pue